In [61]:
import sys

import pandas as pd
import numpy as np
import math
import vowpalwabbit
import json

from matplotlib import pyplot as plt
from analysis import analysis_data
import os.path as osp
from sklearn.model_selection import train_test_split

In [62]:
best_path = osp.join(osp.dirname(analysis_data.__file__), "known_performance.csv")
best_performance = pd.read_csv(best_path)
best_performance.columns = ["agent_id", "timestep", "action", "electrical_storage_soc", "best_consumption", "load", "solar", "carbon", "price"]

In [63]:
best_consumption = best_performance[["electrical_storage_soc", "best_consumption"]]
best_consumption.head()

electrical_storage_soc  best_consumption
0                     0.0      2.188750e+00
1                     0.0      1.009623e-07
2                     0.0      2.819150e+00
3                     0.0      7.714333e-01
4                     0.0      8.511667e-01

In [64]:
context_path = osp.join(osp.dirname(analysis_data.__file__), "consumption_context.csv")
context = pd.read_csv(context_path)

In [65]:
context.columns = ["non_shiftable_load", "solar_generation", "electrical_storage_soc"]
context = context[["non_shiftable_load", "solar_generation"]]
context["electrical_storage_soc"] = best_performance["electrical_storage_soc"]
context.head()

non_shiftable_load  solar_generation  electrical_storage_soc
0        2.188750e+00               0.0                     0.0
1        1.009623e-07               0.0                     0.0
2        2.819150e+00               0.0                     0.0
3        7.714333e-01               0.0                     0.0
4        8.511667e-01               0.0                     0.0

In [66]:
best_consumption = best_performance["best_consumption"]
best_consumption.head()

0    2.188750e+00
1    1.009623e-07
2    2.819150e+00
3    7.714333e-01
4    8.511667e-01
Name: best_consumption, dtype: float64

In [67]:
df = pd.concat([context, best_consumption], axis=1)
df = pd.concat([df]*20, ignore_index=True)
df.head()

non_shiftable_load  solar_generation  electrical_storage_soc  \
0        2.188750e+00               0.0                     0.0   
1        1.009623e-07               0.0                     0.0   
2        2.819150e+00               0.0                     0.0   
3        7.714333e-01               0.0                     0.0   
4        8.511667e-01               0.0                     0.0   

   best_consumption  
0      2.188750e+00  
1      1.009623e-07  
2      2.819150e+00  
3      7.714333e-01  
4      8.511667e-01

In [68]:
argument_string_list = [
                        # "month",  # 0
                        # "day_type",  # 1
                        # "hour",  # 2
                        # "outdoor_dry_bulb",  # 3
                        # "outdoor_dry_bulb_temperature_predicted_6h",  # 4
                        # "outdoor_dry_bulb_temperature_predicted_12h",  # 5
                        # "outdoor_dry_bulb_temperature_predicted_24h",  # 6
                        # "outdoor_relative_humidity",  # 7
                        # "outdoor_relative_humidity_predicted_6h",  # 8
                        # "outdoor_relative_humidity_predicted_12h",  # 9
                        # "outdoor_relative_humidity_predicted_24h",  # 10
                        # "diffuse_solar_irradiance",  # 11
                        # "diffuse_solar_irradiance_predicted_6h",  # 12
                        # "diffuse_solar_irradiance_predicted_12h",  # 13
                        # "diffuse_solar_irradiance_predicted_24h",  # 14
                        # "direct_solar_irradiance",  # 15
                        # "direct_solar_irradiance_predicted_6h",  # 16
                        # "direct_solar_irradiance_predicted_12h",  # 17
                        # "direct_solar_irradiance_predicted_24h",  # 18
                        # "carbon_intensity",  # 19
                        "non_shiftable_load",  # 20
                        "solar_generation",  # 21
                        "electrical_storage_soc",  # 22
                        #"net_electricity_consumption", # 23
                        # "electricity_pricing",  # 24
                        # "electricity_pricing_predicted_6h",  # 25
                        # "electricity_pricing_predicted_12h",  # 26
                        # "electricity_pricing_predicted_24h",  # 27
                        # "action", # 28
                        ]

In [69]:
def get_cost(new_consumption, best_consumption):
    return (new_consumption - best_consumption) ** 2

In [70]:
def to_vw_example_format(local_context, cats_label=None):
    example_dict = {}
    if cats_label is not None:
        chosen_temp, cost, pdf_value = cats_label
        example_dict["_label_ca"] = {
            "action": chosen_temp,
            "cost": cost,
            "pdf_value": pdf_value,
        }

        temp = {}

        for arg in argument_string_list:
            temp[(arg + "={}").format(local_context[arg])] = 1

        example_dict["c"] = temp

    return json.dumps(example_dict)

In [71]:
def predict_action(vw, local_context):
    vw_text_example = to_vw_example_format(local_context)
    return vw.predict(vw_text_example)

In [72]:
df["non_shiftable_load"] = df["non_shiftable_load"] # .shift(-1, fill_value=0)
df["solar_generation"] = df["solar_generation"] # .shift(-1, fill_value=0)
df["electrical_storage_soc"] = df["electrical_storage_soc"] # .shift(-1, fill_value=0)
df.head()

non_shiftable_load  solar_generation  electrical_storage_soc  \
0        2.188750e+00               0.0                     0.0   
1        1.009623e-07               0.0                     0.0   
2        2.819150e+00               0.0                     0.0   
3        7.714333e-01               0.0                     0.0   
4        8.511667e-01               0.0                     0.0   

   best_consumption  
0      2.188750e+00  
1      1.009623e-07  
2      2.819150e+00  
3      7.714333e-01  
4      8.511667e-01

In [73]:
from math import sqrt


def calculate_battery_consumption(current_battery_level, prev_battery_level, efficiency):
    loss_coefficient = 0.006  # potentially 0.006?
    battery_consumption = current_battery_level - prev_battery_level * (1 - loss_coefficient)

    if battery_consumption >= 0:
        battery_consumption = battery_consumption / efficiency
    else:
        battery_consumption = battery_consumption * efficiency

    return battery_consumption


def calculate_battery_level(action_capacity, prev_battery_level, efficiency):
    if action_capacity < -5:
        action_capacity = -5
    elif action_capacity > 5:
        action_capacity = 5

    if action_capacity >= 0:
        new_battery_level = min(prev_battery_level + action_capacity * efficiency, 6.4)
    else:
        new_battery_level = max(0, prev_battery_level + action_capacity / efficiency)
    return new_battery_level


# (action *  = 6.4)
def calculate_efficiency(action_capacity):
    if action_capacity < -5:
        action_capacity = -5.0
    elif action_capacity > 5:
        action_capacity = 5.0

    x = np.abs(action_capacity / 5.0)

    if 0 <= x <= 0.3:
        efficiency = sqrt(0.83)
    elif 0.3 < x < 0.7:
        efficiency = sqrt(0.7775 + 0.175 * x)
    elif 0.7 <= x <= 0.8:  # Optimal efficiency
        efficiency = sqrt(0.9)
    else:
        efficiency = sqrt(1.1 - 0.25 * x)
    return efficiency

In [74]:
df.head()

non_shiftable_load  solar_generation  electrical_storage_soc  \
0        2.188750e+00               0.0                     0.0   
1        1.009623e-07               0.0                     0.0   
2        2.819150e+00               0.0                     0.0   
3        7.714333e-01               0.0                     0.0   
4        8.511667e-01               0.0                     0.0   

   best_consumption  
0      2.188750e+00  
1      1.009623e-07  
2      2.819150e+00  
3      7.714333e-01  
4      8.511667e-01

In [75]:
X_train = df[["non_shiftable_load", "solar_generation", "electrical_storage_soc"]].to_dict(orient="records")
best_consumptions = df["best_consumption"]
loads = df["non_shiftable_load"]
solars = df["solar_generation"]
levels = df["electrical_storage_soc"]

In [ ]:
reward_rate = []
cost_sum = 0.0
iterations = len(X_train)

num_actions = 10
bandwidth = 0.05

# Instantiate VW learner
vw = vowpalwabbit.Workspace(
    "--cats "
    + str(num_actions)
    + "  --bandwidth "
    + str(bandwidth)
    + " --min_value -0.5 --max_value 0.5 --json --chain_hash --coin --epsilon 0.2 -q :: --quiet"
)
consumptions = []
for index, current_context in enumerate(X_train):

    if (index + 1) % 100 == 0:
        print(index + 1)

    predicted_action, pdf_value = predict_action(vw, current_context)

    standard_consumption = loads[index] - solars[index]

    action_capacity = predicted_action * 6.4
    loc_efficiency = calculate_efficiency(predicted_action * 6.4)
    prev_battery_level = levels[index]
    battery_level = calculate_battery_level(action_capacity, prev_battery_level, loc_efficiency)
    battery_consumption = calculate_battery_consumption(battery_level, prev_battery_level, loc_efficiency)

    new_consumption = standard_consumption + battery_consumption
    consumptions.append(new_consumption)
    # print(new_consumption, best_consumptions[index])
    cost = get_cost(new_consumption, best_consumptions[index])

    cost_sum += cost

    txt_ex = to_vw_example_format(context, cats_label=(predicted_action, cost, pdf_value))

    vw_format = vw.parse(txt_ex, vowpalwabbit.LabelType.CONTINUOUS)

    vw.learn(vw_format)

    vw.finish_example(vw_format)

    reward_rate.append(cost_sum / index + 1)

vw.finish()

C:\Users\Team Epoch A\AppData\Local\Temp\ipykernel_8472\2662778412.py:47: RuntimeWarning: invalid value encountered in double_scalars
  reward_rate.append(cost_sum / index + 1)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [ ]:
plt.plot(range(1, iterations + 1), reward_rate)
plt.xlabel("num_iterations", fontsize=14)
plt.ylabel("reward rate", fontsize=14)
plt.title("Plot")
# plt.ylim([0, 1])

plt.show()

print(cost_sum / iterations)


In [ ]:
plt.plot(best_consumptions[:100])

In [ ]:
plt.plot(consumptions[:100])